# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [15]:
#Your code here
import pandas as pd
df = pd.read_csv("homepage_actions.csv")
df

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [16]:
df.action.value_counts()

view     6328
click    1860
Name: action, dtype: int64

In [17]:
view = df.loc[(df.action == "view")]
click = df.loc[df.action == "click"]
vw_clicked = view.set_index("id").join(click.set_index("id"), how="inner", rsuffix="2")
print(f"we see that {len(vw_clicked)} of those who viewed also clicked")
print("all those who clicked also viewed the page")

we see that 1860 of those who viewed also clicked
all those who clicked also viewed the page


In [18]:
v_n_c = len(set(view["id"]) - set(click["id"]))
print(f"{v_n_c} viewed and  didn't click")

4468 viewed and  didn't click


In [19]:
exp = set(df.loc[df.group == "experiment"]["id"])
ctrl = set(df.loc[df.group == "control"]["id"])
print("overlap between experiment and control: ", len(exp&ctrl))

overlap between experiment and control:  0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [20]:
df["count"] = 1
df.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [21]:
control = df[df.group=="control"].pivot(index="id", columns="action", values="count")
control = control.fillna(value=0)

experiment = df[df.group == "experiment"].pivot(index="id", columns="action", values="count")
experiment = experiment.fillna(value=0)

print("experiment average click rate: ", experiment.click.mean())
print("control average click rate: ", control.click.mean())

experiment average click rate:  0.3097463284379172
control average click rate:  0.2797118847539016


In [22]:
#Your code here
#Ho: no differrence between the experimental homepage and control homepage
#Ha: experiment page click rate was higher than control page click rate. 
contingency_table = pd.crosstab(df.group, df.action)
contingency_table

action,click,view
group,,
control,932,3332
experiment,928,2996


In [23]:
experiment_mean = 928/2996
control_mean = 932/3332

In [24]:
import scipy.stats as stats
results = stats.ttest_ind(experiment.click, control.click).pvalue/2
print("p_value: ", results)

p_value:  0.0044125494574791466


In [25]:
""" 
We reject the null hypothesis(pvalue < 0.05) and we see that there is a significant difference between the two
giving indications that that the experimental page was more efective than the control page. 
"""

' \nWe reject the null hypothesis(pvalue < 0.05) and we see that there is a significant difference between the two\ngiving indications that that the experimental page was more efective than the control page. \n'

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [26]:
#Your code here
exp_expected = control_mean * len(experiment)
exp_expected

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [27]:
#Your code here
import numpy as np
n = len(experiment)
p = control_mean
var = n * p *(1 - p)
sd = np.sqrt(var)
sd

24.568547907005815

In [28]:
z = (experiment.click.sum() - exp_expected) / sd
z

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [29]:
#Your code here
import scipy.stats as stats
1 - stats.norm.cdf(z)

0.00012486528006949715

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [30]:
""" 
all are lower than the pvalue and hence reject the null hypothesis
giving indications that that the experimental page was more efective than the control page.
"""

' \nall are lower than the pvalue and hence reject the null hypothesis\ngiving indications that that the experimental page was more efective than the control page.\n'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.